In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from scipy.stats import poisson, uniform, expon, pareto
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

In [29]:
p0, p0p, p1, p0pp = sp.symbols('p_0 p_{00} p_1 p_{000}')
a, b, x, g, p, E  = sp.symbols('a b x g p E')

In [30]:
f0p0high = p1 / (1-sp.exp(-b)) * sp.integrate(sp.exp(-x), (x, b, sp.oo))
f0p0high

p_1*exp(-b)/(1 - exp(-b))

In [41]:
f0p0high.evalf(subs={a:0.3, b:1.})

0.581976706869326*p_1

In [39]:
integrand = p1 * sp.exp(-x) * (sp.exp(-b+x) + a*(1-sp.exp(-b+x)) + (1-a)*(1-sp.exp(-b+x))*(a+(1-a)*g)) / (1-sp.exp(-b))
f0p0low = sp.integrate(integrand, (x,0,b))
f0p0low

b*(-a**2*g*p_1 + a**2*p_1 + 2*a*g*p_1 - 2*a*p_1 - g*p_1 + p_1)/(exp(b) - 1) + Piecewise((-(-a**2*g*p_1*exp(b) + a**2*p_1*exp(b) + 2*a*g*p_1*exp(b) - 2*a*p_1*exp(b) - g*p_1*exp(b))/(exp(b) - 1) + (-a**2*g*p_1*exp(b) + a**2*p_1*exp(b) + 2*a*g*p_1*exp(b) - 2*a*p_1*exp(b) - g*p_1*exp(b))*exp(-b)/(exp(b) - 1), Ne(b, 0)), (b*(-(-a**2*g*p_1 + a**2*p_1 + 2*a*g*p_1 - 2*a*p_1 - g*p_1 + p_1)/(exp(b) - 1) + (a**2*g*p_1*exp(b) - a**2*g*p_1 - a**2*p_1*exp(b) + a**2*p_1 - 2*a*g*p_1*exp(b) + 2*a*g*p_1 + 2*a*p_1*exp(b) - 2*a*p_1 + g*p_1*exp(b) - g*p_1 + p_1)/(exp(b) - 1)), True))

In [40]:
f0p0low.evalf(subs={a:0.3, b:1.})

0.20483141363403*g*p_1 + 0.79516858636597*p_1

In [32]:
integrand

p_1*(a*(1 - exp(-b + x)) + (1 - a)*(1 - exp(-b + x))*(a + g*(1 - a)) + exp(-b + x))*exp(-x)/(1 - exp(-b))

In [33]:
f0pp0p     = a*(1-a)*p1
f0ppp0pp   = a**2*p1
f1p1       = a*(2-a)*p1
fRestpRest = p1*2*(1-a)*a**2 / ((1-a)*(1-2*a))
fRestpRest

2*a**2*p_1/(1 - 2*a)

In [43]:
sp.simplify(f0pp0p + f0ppp0pp + f1p1 + fRestpRest)

a*p_1*(-2*a**2 + 5*a - 3)/(2*a - 1)

In [34]:
fRestpRest.evalf(subs={})

2.0*a**2*p_1/(1.0 - 2.0*a)

In [35]:
full = f0p0low + f0p0high + f0pp0p + f0ppp0pp + f1p1 + fRestpRest
full = sp.simplify(full)
full

Piecewise((-p_1*(2*a**2*(exp(b) - 1) + (2*a - 1)*(b*(a**2*g - a**2 - 2*a*g + 2*a + g - 1) - 1) - (2*a - 1)*(exp(b) - 1)*(a**2*g - 2*a*g - a*(a - 2) - a*(a - 1) + 2*a + g))/((2*a - 1)*(exp(b) - 1)), (b > 0) | (b < 0)), (-p_1*(2*a**2*(exp(b) - 1) + (2*a - 1)*(b*(a**2*g - a**2 - 2*a*g + 2*a + g - 1) - 1) + (2*a - 1)*(a*(1 - exp(b))*(3 - a) - b*(a**2*g - a**2 - 2*a*g + 2*a + g)*exp(b)))/((2*a - 1)*(exp(b) - 1)), True))

In [36]:
p0pp  = a*p1
p0p   = (1-a)*p1
p0    = p1 / (a*(1-sp.exp(-b)))
pRest = (1-a)*a/((1-a)*(1-2*a)) * p1

In [37]:
p1solved = sp.solve(sp.Eq(p0 + p0p + p0pp + p1 + pRest, 1), p1)[0]
p1solved

a*(2*a*exp(b) - 2*a - exp(b) + 1)/(3*a**2*exp(b) - 3*a**2 + 2*a - exp(b))

In [38]:
p1solved.evalf(subs={a:0.3, b:1.})

0.124637683091749

In [15]:
final = sp.simplify(full.evalf(subs={p1:p1solved}))
final

Piecewise((a*(-2.0*a**2*(exp(b) - 1) - (2.0*a - 1.0)*(b*(a**2*g - a**2 - 2.0*a*g + 2.0*a + g - 1.0) - 1.0) + (2.0*a - 1.0)*(exp(b) - 1.0)*(a**2*g - 2.0*a*g - a*(a - 2.0) - a*(a - 1.0) + 2.0*a + g))*(2.0*a*exp(b) - 2.0*a - exp(b) + 1.0)/((2.0*a - 1.0)*(exp(b) - 1.0)*(3.0*a**2*exp(b) - 3.0*a**2 + 2.0*a - exp(b))), (b > 0) | (b < 0)), (-a*(2.0*a**2*(exp(b) - 1) + (2.0*a - 1.0)*(b*(a**2*g - a**2 - 2.0*a*g + 2.0*a + g - 1.0) - 1.0) + (2.0*a - 1.0)*(a*(a - 3.0)*(exp(b) - 1.0) - b*(a**2*g - a**2 - 2.0*a*g + 2.0*a + g)*exp(b)))*(2.0*a*exp(b) - 2.0*a - exp(b) + 1.0)/((2.0*a - 1.0)*(exp(b) - 1.0)*(3.0*a**2*exp(b) - 3.0*a**2 + 2.0*a - exp(b))), True))

In [16]:
temp_matts_lambda = sp.lambdify((a, b, g), final)
temp_matts_lambda

<function _lambdifygenerated(a, b, g)>

In [22]:
# compare
a, b, g = 0.3, 1., 0.0
temp_matts_lambda(a=0.3, b=0.1, g=0.0)

array(0.31087698)